# Horn of Africa initial basic accessibility

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import split, unary_union
from shapely.geometry import box, Point

import matplotlib.pyplot as plt

C:\Users\war-machine\anaconda3\envs\gostnets3_8\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [4]:
# Get reference to GOSTNets
sys.path.append(r'../../../../GOSTnets')
import GOSTnets as gn

In [5]:
from GOSTnets.load_osm import *

## Create road network
Countries for this analysis include Ethiopia, Kenya, Somalia, Djibouti, South Sudan, and Sudan. Initially the OSM pbf files were downloaded from Geofabrik for these countries then merged together using the Osmium command line tool. Unfortunately this resulted in the all the roads being able to be imported. Instead the whole OSM Planet file was downloaded, then Osmium was used to extract only roads within the horn of africa spatial extent.

In [5]:
import time
start_time = time.time()

In [6]:
f = r'D:\data\planet-210614-clipped-highways2.osm.pbf'

In [7]:
# create OSM_to_network object from the load_osm GOSTnets sub-module
horn_of_africa = OSM_to_network(f)

C:\Users\war-machine\anaconda3\envs\gostnets3_8\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
print(f"sec elapsed: {time.time() - start_time}")

sec elapsed: 216.4989528656006


In [9]:
# show the different highway types and counts
horn_of_africa.roads_raw.infra_type.value_counts()

residential       887026
path              323267
unclassified      284615
track             278295
service            53861
tertiary           22731
footway            14141
secondary          10893
primary             7649
trunk               5882
road                1809
trunk_link          1016
primary_link         916
secondary_link       780
living_street        589
construction         524
tertiary_link        478
pedestrian           439
steps                188
motorway             115
motorway_link         83
bridleway             55
cycleway              42
yes                    7
platform               6
proposed               3
raceway                3
corridor               2
bus_guideway           1
rest_area              1
rural road             1
d                      1
Name: infra_type, dtype: int64

## Decided to filter the primary roads

In [10]:
#accepted_road_types = ['tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link']

In [11]:
#horn_of_africa.filterRoads(acceptedRoads = accepted_road_types)

In [12]:
#horn_of_africa.roads_raw.infra_type.value_counts()

In [13]:
# load_osm GOSTnets sub-module intermediate step
horn_of_africa.generateRoadsGDF(verbose = False)

In [14]:
print(f"sec elapsed: {time.time() - start_time}")

sec elapsed: 29107.65540409088


In [15]:
# load_osm GOSTnets sub-module final step, creates the graph
horn_of_africa.initialReadIn()

In [16]:
print(nx.info(horn_of_africa.network))

Name: 
Type: MultiDiGraph
Number of nodes: 3361911
Number of edges: 4584167
Average in degree:   1.3636
Average out degree:   1.3636


In [17]:
print(f"sec elapsed: {time.time() - start_time}")

sec elapsed: 29442.828305721283


### save the graph, this creates a pickle which can be imported layer, it also saves the nodes and the edges as CSVs, which can be opened in QGIS

In [18]:
gn.save(horn_of_africa.network,"horn_of_africa_unclean5_full",r"temp")

In [5]:
# open horn_of_africa_unclean
G = nx.read_gpickle(os.path.join(r'temp', 'horn_of_africa_unclean5_full.pickle'))

In [10]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 3361911
Number of edges: 4584167
Average in degree:   1.3636
Average out degree:   1.3636


In [6]:
horn_of_africa_UTMZ = {'init': 'epsg:32638'}

WGS = {'init': 'epsg:4326'} # do not adjust. OSM natively comes in ESPG 4326

### clean network processes, including simplifying edges between intersections

In [21]:
# print('start: %s\n' % time.ctime())
# G_clean = gn.clean_network(G, UTM = horn_of_africa_UTMZ, WGS = {'init': 'epsg:4326'}, junctdist = 10, verbose = False)

# print('\nend: %s' % time.ctime())
# print('\n--- processing complete')

start: Fri Jul  9 03:36:59 2021



C:\Users\war-machine\anaconda3\envs\gostnets3_8\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\war-machine\anaconda3\envs\gostnets3_8\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
../../../../GOSTnets\GOSTnets\core.py:1426: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a project

4277282


KeyboardInterrupt: 

In [ ]:
# let's print info on our clean version
print(nx.info(G_clean))

In [ ]:
# save and inspect graph
gn.save(G_clean,"G_cleaned_horn_of_africa5",r"temp")

### Identify only the largest graph

In [11]:
# compatible with NetworkX 2.4
list_of_subgraphs = list(G.subgraph(c).copy() for c in nx.weakly_connected_components(G))
max_graph = None
max_edges = 0

# To sort the list in place...
list_of_subgraphs.sort(key=lambda x: x.number_of_edges(), reverse=True)

In [12]:

for subgraph in list_of_subgraphs:
    print(len(subgraph))
    # to inspect
    #gn.save(subgraph,f"{len(subgraph)}_inspect",r"temp", pickle = False, edges = True, nodes = False)

3258104
1633
1010
967
1129
755
647
654
545
468
479
374
367
389
384
345
275
268
292
242
273
194
207
247
224
245
160
187
192
173
153
150
168
153
143
152
146
139
143
142
124
129
125
134
125
125
113
118
137
117
99
112
112
118
109
105
84
112
114
118
91
99
81
90
90
99
102
96
83
89
90
94
83
107
81
79
86
95
48
83
87
78
73
88
71
67
86
70
80
69
65
68
73
59
74
79
76
71
85
67
70
78
70
74
74
77
77
62
71
68
65
68
57
63
66
73
66
60
67
69
60
61
72
54
63
69
61
63
61
66
54
62
60
64
50
57
62
49
54
57
53
53
62
55
46
68
49
51
57
60
59
52
60
62
60
48
54
52
50
55
59
46
57
50
48
48
57
53
63
54
51
54
53
54
49
49
52
58
50
49
44
51
47
50
58
50
51
48
41
48
54
45
51
51
46
49
49
57
39
44
45
45
52
43
50
49
55
54
45
48
46
53
41
41
53
51
43
43
49
37
40
40
43
42
40
41
49
48
40
43
44
49
44
44
46
35
39
46
43
43
30
34
35
42
44
40
40
45
45
35
36
39
39
44
41
40
40
41
40
40
37
38
34
43
31
36
37
39
38
40
32
36
40
34
38
33
37
35
39
37
38
39
38
37
30
33
40
38
38
35
35
40
38
33
39
24
39
40
38
40
37
40
37
30
30
24
36
34
32
31
31


4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
1
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2


In [14]:
G_largest = list_of_subgraphs[0]

In [15]:
len(G_largest)

3258104

In [7]:
# # compatible with NetworkX 2.4
# list_of_subgraphs = list(G.subgraph(c).copy() for c in nx.strongly_connected_components(G))
# max_graph = None
# max_edges = 0
# for i in list_of_subgraphs:
#     if i.number_of_edges() > max_edges:
#         max_edges = i.number_of_edges()
#         max_graph = i

# # set your graph equal to the largest sub-graph
# G_largest = max_graph

In [8]:
# print info about the largest sub-graph
print(nx.info(G_largest))

Name: 
Type: MultiDiGraph
Number of nodes: 197517
Number of edges: 314002
Average in degree:   1.5897
Average out degree:   1.5897


In [13]:
len(G_largest)

197517

In [16]:
# save and inspect graph
gn.save(G_largest,"G_largest_cleaned_horn_of_africa5",r"temp")

In [6]:
# load graph
G = nx.read_gpickle(os.path.join(r'temp', 'G_largest_cleaned_horn_of_africa5.pickle'))

## insert origins
Origins are from the GHS SMOD rural categories, and they were converted to vector points in QGIS.

In [7]:
origins = gpd.read_file(r'C:\Users\war-machine\Documents\world_bank_work\horn_of_africa_analysis\horn_of_africa_ghs_rural_pts_4326.shp')

In [8]:
origins['osmid'] = 1110000000 + origins.index
origins

,VALUE,geometry,osmid
0,11.0,POINT (35.60569 23.13432),1110000000
1,11.0,POINT (35.61628 23.13432),1110000001
2,11.0,POINT (35.59510 23.12619),1110000002
3,11.0,POINT (35.60569 23.12619),1110000003
4,11.0,POINT (35.61628 23.12619),1110000004
...,...,...,...
4783432,11.0,POINT (42.68058 -0.19391),1114783432
4783433,11.0,POINT (42.69117 -0.19391),1114783433
4783434,11.0,POINT (42.66999 -0.20204),1114783434
4783435,11.0,POINT (42.68058 -0.20204),1114783435


In [9]:
# find graph utm zone
G_utm = gn.utm_of_graph(G)
G_utm

'+proj=utm +zone=37 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'

### snap origins to the graph (snaps to the nearest node on the graph)

In [10]:
%%time
#no need to do advanced_snap at this extent
G2, pois_meter, new_footway_edges = gn.advanced_snap(G, origins, u_tag = 'stnode', v_tag = 'endnode', node_key_col='node_ID', poi_key_col='osmid', path=None, threshold=2000, measure_crs=G_utm, factor=1000)

Building rtree...
Projecting POIs to the network...
Updating internal nodes...
print _new_nodes
                               geometry        highway     node_ID
0        POINT (147134.734 2556700.032)  projected_pap  9990000000
1        POINT (147741.866 2556148.256)  projected_pap  9990000001
2        POINT (146964.436 2556854.802)  projected_pap  9990000002
3        POINT (147571.645 2556302.957)  projected_pap  9990000003
4        POINT (148178.810 2555751.152)  projected_pap  9990000004
...                                 ...            ...         ...
2875550   POINT (910149.555 -20133.463)  projected_pap  9992875550
2875551   POINT (911028.557 -20360.270)  projected_pap  9992875551
2875552   POINT (911449.612 -20233.754)  projected_pap  9992875552
2875553   POINT (908690.654 -22286.585)  projected_pap  9992875553
2875554   POINT (908632.975 -23310.722)  projected_pap  9992875554

[2875555 rows x 3 columns]
Updating internal edges...
Updating external links...
print unvalid line

In [9]:
#%%time
#no need to do advanced_snap at this extent
#G2, pois_meter, new_footway_edges = gn.advanced_snap(G, origins, u_tag = 'stnode', v_tag = 'endnode', node_key_col='node_ID', poi_key_col='osmid', path=None, threshold=2000, measure_crs=G_utm)
#snapped_origins = gn.pandana_snap(G, origins, source_crs = 'epsg:4326', target_crs = G_utm)

Building rtree...
Projecting POIs to the network...
Updating internal nodes...
print _new_nodes
                               geometry        highway     node_ID
0        POINT (147134.734 2556700.032)  projected_pap  9990000000
1        POINT (147741.866 2556148.256)  projected_pap  9990000001
2        POINT (146964.436 2556854.802)  projected_pap  9990000002
3        POINT (147571.645 2556302.957)  projected_pap  9990000003
4        POINT (148178.810 2555751.152)  projected_pap  9990000004
...                                 ...            ...         ...
2875550   POINT (910149.555 -20133.463)  projected_pap  9992875550
2875551   POINT (911028.557 -20360.270)  projected_pap  9992875551
2875552   POINT (911449.612 -20233.754)  projected_pap  9992875552
2875553   POINT (908690.654 -22286.585)  projected_pap  9992875553
2875554   POINT (908632.975 -23310.722)  projected_pap  9992875554

[2875555 rows x 3 columns]
Updating internal edges...
Updating external links...
print unvalid line

In [11]:
new_footway_edges

,osmid,geometry,oneway,highway,length,stnode,endnode,node_ID
7714124,1110000000,"LINESTRING (35.60569 23.13432, 35.55617 23.08148)",False,projected_footway,7.753087,1110000000,9990000000,9990000000_1110000000
7714125,1110000001,"LINESTRING (35.61628 23.13432, 35.56221 23.07663)",False,projected_footway,8.464558,1110000001,9990000001,9990000001_1110000001
7714126,1110000002,"LINESTRING (35.59510 23.12619, 35.55447 23.08284)",False,projected_footway,6.360789,1110000002,9990000002,9990000002_1110000002
7714127,1110000003,"LINESTRING (35.60569 23.12619, 35.56052 23.07799)",False,projected_footway,7.072257,1110000003,9990000003,9990000003_1110000003
7714128,1110000004,"LINESTRING (35.61628 23.12619, 35.56656 23.07314)",False,projected_footway,7.783777,1110000004,9990000004,9990000004_1110000004
...,...,...,...,...,...,...,...,...
12497556,1114783432,"LINESTRING (42.68058 -0.19391, 42.67645 -0.20032)",False,projected_footway,0.846762,1114783432,1795854,1795854_1114783432
12497557,1114783433,"LINESTRING (42.69117 -0.19391, 42.68790 -0.19020)",False,projected_footway,0.549150,1114783433,1446594,1446594_1114783433
12497558,1114783434,"LINESTRING (42.66999 -0.20204, 42.67029 -0.20122)",False,projected_footway,0.097066,1114783434,9992875553,9992875553_1114783434
12497559,1114783435,"LINESTRING (42.68058 -0.20204, 42.67645 -0.20032)",False,projected_footway,0.498605,1114783435,1795854,1795854_1114783435


In [15]:
new_footway_edges

,osmid,geometry,oneway,highway,length,stnode,endnode,node_ID
7714144,1110000020,"LINESTRING (35.56332 23.09367, 35.55373 23.08343)",False,projected_footway,1502.648291,1110000020,9990000020,9990000020_1110000020
7714153,1110000029,"LINESTRING (35.56332 23.08555, 35.55808 23.07995)",False,projected_footway,821.646668,1110000029,9990000029,9990000029_1110000029
7714154,1110000030,"LINESTRING (35.57391 23.08555, 35.56412 23.07510)",False,projected_footway,1533.201342,1110000030,9990000030,9990000030_1110000030
7714162,1110000038,"LINESTRING (35.56332 23.07742, 35.56242 23.07646)",False,projected_footway,140.650241,1110000038,9990000038,9990000038_1110000038
7714163,1110000039,"LINESTRING (35.57391 23.07742, 35.56847 23.07161)",False,projected_footway,852.254244,1110000039,9990000039,9990000039_1110000039
...,...,...,...,...,...,...,...,...
12497556,1114783432,"LINESTRING (42.68058 -0.19391, 42.67645 -0.20032)",False,projected_footway,846.762249,1114783432,1795854,1795854_1114783432
12497557,1114783433,"LINESTRING (42.69117 -0.19391, 42.68790 -0.19020)",False,projected_footway,549.150483,1114783433,1446594,1446594_1114783433
12497558,1114783434,"LINESTRING (42.66999 -0.20204, 42.67029 -0.20122)",False,projected_footway,97.065607,1114783434,9992875553,9992875553_1114783434
12497559,1114783435,"LINESTRING (42.68058 -0.20204, 42.67645 -0.20032)",False,projected_footway,498.605304,1114783435,1795854,1795854_1114783435


## Snap destinations to the road graph
Destinations are centroids created in QGIS of the GHS urban centers

In [12]:
# insert destinations
destinations = gpd.read_file(r'C:\Users\war-machine\Documents\world_bank_work\horn_of_africa_analysis\horn_of_africa_ghs_stat_centroids2.shp')

In [13]:
destinations

,fid,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE901,SDG_A2G14,SDG_OS15MX,geometry
0,3475.0,3475.0,1.0,27.0,13.415981,22.420601,13.473200,22.482918,13.441260,22.448188,...,0.0,0.0,0.000000,0.0,available,6.77122,24.7727,0.006136,64.07,POINT (22.44819 13.44126)
1,3478.0,3478.0,2.0,3.0,13.800267,22.493902,13.816625,22.514710,13.807082,22.502775,...,0.0,0.0,0.000000,0.0,available,7.84638,-29.389,0.000000,NaN,POINT (22.50278 13.80708)
2,3484.0,3484.0,1.0,9.0,12.109492,22.581877,12.142124,22.613938,12.122635,22.599989,...,0.0,0.0,0.000000,0.0,available,10.27490,-5.2497,0.000000,78.56,POINT (22.59999 12.12264)
3,3487.0,3487.0,1.0,10.0,15.011973,22.783617,15.044757,22.825476,15.026726,22.801921,...,0.0,0.0,0.000000,0.0,available,5.91611,0.52879,0.000000,NaN,POINT (22.80192 15.02673)
4,3494.0,3494.0,1.0,3.0,12.942079,22.870255,12.966583,22.881783,12.954331,22.876018,...,0.0,0.0,0.000000,0.0,available,8.35383,-16.2193,0.000000,NaN,POINT (22.87602 12.95433)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,5839.0,5839.0,0.0,94.0,4.360873,47.389311,4.506618,47.512125,4.428748,47.448015,...,0.0,0.0,0.000000,0.0,available,5.27807,0.014611,0.685121,NaN,POINT (47.44802 4.42875)
832,5849.0,5849.0,2.0,4.0,10.707730,48.324559,10.724015,48.346337,10.715872,48.335448,...,0.0,0.0,0.022302,3.0,available,10.93800,0.11286,0.000000,NaN,POINT (48.33545 10.71587)
833,5861.0,5861.0,1.0,9.0,8.390332,48.463480,8.414696,48.505537,8.402514,48.486182,...,0.0,0.0,0.000000,0.0,available,7.56884,1.8773,0.000000,54.22,POINT (48.48618 8.40251)
834,5871.0,5871.0,1.0,16.0,11.245304,49.159866,11.294193,49.201985,11.275350,49.184212,...,0.0,0.0,0.033539,3.0,available,5.40898,0.05113,0.000000,62.94,POINT (49.18421 11.27535)


In [14]:
snapped_destinations = gn.pandana_snap(G, destinations, source_crs = 'epsg:4326', target_crs = G_utm)

In [15]:
snapped_destinations

,fid,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,...,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE901,SDG_A2G14,SDG_OS15MX,geometry,NN,NN_dist
0,3475.0,3475.0,1.0,27.0,13.415981,22.420601,13.473200,22.482918,13.441260,22.448188,...,0.000000,0.0,available,6.77122,24.7727,0.006136,64.07,POINT (22.44819 13.44126),675113,25.499324
1,3478.0,3478.0,2.0,3.0,13.800267,22.493902,13.816625,22.514710,13.807082,22.502775,...,0.000000,0.0,available,7.84638,-29.389,0.000000,NaN,POINT (22.50278 13.80708),2408655,9409.866250
2,3484.0,3484.0,1.0,9.0,12.109492,22.581877,12.142124,22.613938,12.122635,22.599989,...,0.000000,0.0,available,10.27490,-5.2497,0.000000,78.56,POINT (22.59999 12.12264),719629,86378.952896
3,3487.0,3487.0,1.0,10.0,15.011973,22.783617,15.044757,22.825476,15.026726,22.801921,...,0.000000,0.0,available,5.91611,0.52879,0.000000,NaN,POINT (22.80192 15.02673),2016197,301.037874
4,3494.0,3494.0,1.0,3.0,12.942079,22.870255,12.966583,22.881783,12.954331,22.876018,...,0.000000,0.0,available,8.35383,-16.2193,0.000000,NaN,POINT (22.87602 12.95433),190369,62.089054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,5839.0,5839.0,0.0,94.0,4.360873,47.389311,4.506618,47.512125,4.428748,47.448015,...,0.000000,0.0,available,5.27807,0.014611,0.685121,NaN,POINT (47.44802 4.42875),1828145,1865.893882
832,5849.0,5849.0,2.0,4.0,10.707730,48.324559,10.724015,48.346337,10.715872,48.335448,...,0.022302,3.0,available,10.93800,0.11286,0.000000,NaN,POINT (48.33545 10.71587),2059544,71.380626
833,5861.0,5861.0,1.0,9.0,8.390332,48.463480,8.414696,48.505537,8.402514,48.486182,...,0.000000,0.0,available,7.56884,1.8773,0.000000,54.22,POINT (48.48618 8.40251),1140390,57.029235
834,5871.0,5871.0,1.0,16.0,11.245304,49.159866,11.294193,49.201985,11.275350,49.184212,...,0.033539,3.0,available,5.40898,0.05113,0.000000,62.94,POINT (49.18421 11.27535),805785,29.097550


In [16]:
destinationNodes = list(snapped_destinations['NN'].unique())

In [40]:
destinationNodes

[675113,
 2408655,
 719629,
 2016197,
 190369,
 1673268,
 1057282,
 29312,
 398428,
 3167713,
 771534,
 1108622,
 2992836,
 1798588,
 703595,
 1559630,
 2336334,
 2469733,
 1969423,
 240518,
 525930,
 1858321,
 37868,
 2743667,
 1762935,
 3159706,
 2748544,
 900689,
 1452335,
 80249,
 2456793,
 1466397,
 1971041,
 1205427,
 300172,
 3031156,
 2661544,
 272714,
 1440994,
 2855806,
 725196,
 2873212,
 2252942,
 3356234,
 3262160,
 2355893,
 2426922,
 1627220,
 1878929,
 2132312,
 3256714,
 2721132,
 2187610,
 3047336,
 1517473,
 1552264,
 3054939,
 14597,
 2976353,
 2237112,
 3154150,
 2245253,
 1993320,
 2844019,
 3032384,
 351387,
 856972,
 1468033,
 927633,
 1873760,
 2880754,
 70816,
 384286,
 1589640,
 2231626,
 1213612,
 284358,
 53707,
 2263503,
 1655892,
 1912560,
 1196552,
 1697514,
 2040740,
 875309,
 141906,
 2483280,
 1599770,
 3230526,
 1013744,
 3071336,
 2606183,
 2674781,
 3342475,
 3272264,
 2898142,
 3067296,
 3358580,
 2451105,
 2920209,
 1529753,
 2439725,
 61495,
 29

In [37]:
import pickle

In [38]:
with open("destinationNodes_horn_of_africa_6.txt", "wb") as fp:   #Pickling
    pickle.dump(destinationNodes, fp)

In [18]:
gn.example_edge(G2)

(0, 2394945, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000170347C02E0>, 'key': 'edge_600631', 'infra_type': 'unclassified', 'id': 600631.0, 'Wkt': <shapely.geometry.linestring.LineString object at 0x00000162EE154DC0>, 'length': 0.01803608684586798, 'osm_id': '231698135', 'oneway': True, 'node_ID': nan, 'osmid': nan, 'highway': nan})


### add time to the graph edges as an attribute

In [ ]:
# for u, v, data in G2.edges(data=True):

#         orig_len = data[distance_tag] * factor

#         # Note that this is a MultiDiGraph so there could
#         # be multiple indices here, I naively assume this is not
#         # the case
#         data['length'] = orig_len

#         # get appropriate speed limit
#         if graph_type == 'walk':

In [19]:
%%time
# note that the length above is in km, therefore set factor to 1000
G2_time = gn.convert_network_to_time(G2, 'length', road_col = 'infra_type', factor = 1000)

Wall time: 15min 50s


In [ ]:
## Note to improve: Pandana advanced snap calculates the length for new lines, but it is in meters, and also Euclidean distance distance. Fix this?

In [20]:
gn.example_edge(G2)

(0, 2394945, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000170347C02E0>, 'key': 'edge_600631', 'infra_type': 'unclassified', 'id': 600631.0, 'Wkt': <shapely.geometry.linestring.LineString object at 0x00000162EE154DC0>, 'length': 0.01803608684586798, 'osm_id': '231698135', 'oneway': True, 'node_ID': nan, 'osmid': nan, 'highway': nan})


## calculate OD matrix

In [21]:
print(nx.info(G2))

Name: 
Type: MultiDiGraph
Number of nodes: 10917096
Number of edges: 16943978
Average in degree:   1.5521
Average out degree:   1.5521


In [13]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 3258104
Number of edges: 4488351
Average in degree:   1.3776
Average out degree:   1.3776


In [22]:
# save and inspect graph
gn.save(G2,"G_largest_horn_of_africa6_adv_snap",r"temp")

In [24]:
pois_meter

,VALUE,geometry,osmid,near_idx,near_lines,kne_idx,pp,pp_id
0,11.0,POINT (35.60569 23.13432),1110000000,"[590720, 596707, 2462649, 2462650, 1271747]","590720 LINESTRING (146257.037 2557497.702,...",590720,POINT (147134.733873912 2556700.031761044),9.990000e+09
1,11.0,POINT (35.61628 23.13432),1110000001,"[590720, 596707, 2462649, 2462650, 1271747]","590720 LINESTRING (146257.037 2557497.702,...",590720,POINT (147741.8664053975 2556148.256253783),9.990000e+09
2,11.0,POINT (35.59510 23.12619),1110000002,"[590720, 596707, 2462649, 2462650, 1271747]","590720 LINESTRING (146257.037 2557497.702,...",590720,POINT (146964.4358050937 2556854.8024222),9.990000e+09
3,11.0,POINT (35.60569 23.12619),1110000003,"[590720, 596707, 2462649, 2462650, 1271747]","590720 LINESTRING (146257.037 2557497.702,...",590720,POINT (147571.6449673797 2556302.957271169),9.990000e+09
4,11.0,POINT (35.61628 23.12619),1110000004,"[590720, 596707, 2462649, 2462650, 1271747]","590720 LINESTRING (146257.037 2557497.702,...",590720,POINT (148178.8101277968 2555751.152110011),9.990000e+09
...,...,...,...,...,...,...,...,...
4783432,11.0,POINT (42.68058 -0.19391),1114783432,"[2075061, 213809, 2394677, 2394676, 1116055]","2075061 LINESTRING (908740.148 -21820.294, ...",2075061,POINT (909376.7703161587 -22187.69092819558),NaN
4783433,11.0,POINT (42.69117 -0.19391),1114783433,"[213809, 213810, 3492378, 1407800, 818985]","213809 LINESTRING (910435.885 -20334.426, ...",213809,POINT (910654.6637628503 -21066.78949012153),NaN
4783434,11.0,POINT (42.66999 -0.20204),1114783434,"[1116055, 2075061, 1116056, 2394677, 2394676]","1116055 LINESTRING (908166.677 -22091.204, ...",1116055,POINT (908690.6543883956 -22286.58529558804),NaN
4783435,11.0,POINT (42.68058 -0.20204),1114783435,"[2394677, 2394676, 2075061, 1116055, 2841275]","2394677 LINESTRING (909376.770 -22187.691, ...",2394677,POINT (909376.7703161587 -22187.69092819558),NaN


In [25]:
pois_meter2 = pois_meter[['osmid','VALUE','geometry']]

In [26]:
pois_meter2

,osmid,VALUE,geometry
0,1110000000,11.0,POINT (35.60569 23.13432)
1,1110000001,11.0,POINT (35.61628 23.13432)
2,1110000002,11.0,POINT (35.59510 23.12619)
3,1110000003,11.0,POINT (35.60569 23.12619)
4,1110000004,11.0,POINT (35.61628 23.12619)
...,...,...,...
4783432,1114783432,11.0,POINT (42.68058 -0.19391)
4783433,1114783433,11.0,POINT (42.69117 -0.19391)
4783434,1114783434,11.0,POINT (42.66999 -0.20204)
4783435,1114783435,11.0,POINT (42.68058 -0.20204)


In [57]:
G2.nodes[1110000001]

{'x': 35.616277888398876,
 'y': 23.13431505024561,
 'geometry': <shapely.geometry.point.Point at 0x17933a23790>,
 'highway': 'poi'}

In [58]:
G2.nodes[1110000004]

{'x': 35.616277888398876,
 'y': 23.126186748736846,
 'geometry': <shapely.geometry.point.Point at 0x17933a23820>,
 'highway': 'poi'}

In [59]:
nx.shortest_path(G2,1110000001,1110000004)

[1110000001,
 9990000001,
 9990000038,
 9990000010,
 9990000030,
 9990000006,
 9990000060,
 9990000022,
 9990000004,
 1110000004]

In [61]:
nx.shortest_path_length(G2,1110000001,1110000004, weight='length')

16.838767661009058

In [62]:
destinationNodes

[675113,
 2408655,
 719629,
 2016197,
 190369,
 1673268,
 1057282,
 29312,
 398428,
 3167713,
 771534,
 1108622,
 2992836,
 1798588,
 703595,
 1559630,
 2336334,
 2469733,
 1969423,
 240518,
 525930,
 1858321,
 37868,
 2743667,
 1762935,
 3159706,
 2748544,
 900689,
 1452335,
 80249,
 2456793,
 1466397,
 1971041,
 1205427,
 300172,
 3031156,
 2661544,
 272714,
 1440994,
 2855806,
 725196,
 2873212,
 2252942,
 3356234,
 3262160,
 2355893,
 2426922,
 1627220,
 1878929,
 2132312,
 3256714,
 2721132,
 2187610,
 3047336,
 1517473,
 1552264,
 3054939,
 14597,
 2976353,
 2237112,
 3154150,
 2245253,
 1993320,
 2844019,
 3032384,
 351387,
 856972,
 1468033,
 927633,
 1873760,
 2880754,
 70816,
 384286,
 1589640,
 2231626,
 1213612,
 284358,
 53707,
 2263503,
 1655892,
 1912560,
 1196552,
 1697514,
 2040740,
 875309,
 141906,
 2483280,
 1599770,
 3230526,
 1013744,
 3071336,
 2606183,
 2674781,
 3342475,
 3272264,
 2898142,
 3067296,
 3358580,
 2451105,
 2920209,
 1529753,
 2439725,
 61495,
 29

In [63]:
G2.nodes[2469733]

{'x': 24.5788061,
 'y': 10.948403799999998,
 'geometry': <shapely.geometry.point.Point at 0x1790d6204f0>,
 'highway': nan}

In [64]:
nx.shortest_path_length(G2,1110000001,2469733, weight='length')

NetworkXNoPath: Node 2469733 not reachable from 1110000001

In [35]:
# save new_nodes
pois_meter2.to_csv(r"temp/clipped_origin_nodes_horn_of_africa6.csv")

In [29]:
pois_meter2['osmid'] = pd.to_numeric(pois_meter2['osmid'])
pois_meter2['VALUE'] = pd.to_numeric(pois_meter2['VALUE'])

In [30]:
originNodes_list = list(pois_meter2['osmid'])

In [31]:
originNodes_list

[1110000000,
 1110000001,
 1110000002,
 1110000003,
 1110000004,
 1110000005,
 1110000006,
 1110000007,
 1110000008,
 1110000009,
 1110000010,
 1110000011,
 1110000012,
 1110000013,
 1110000014,
 1110000015,
 1110000016,
 1110000017,
 1110000018,
 1110000019,
 1110000020,
 1110000021,
 1110000022,
 1110000023,
 1110000024,
 1110000025,
 1110000026,
 1110000027,
 1110000028,
 1110000029,
 1110000030,
 1110000031,
 1110000032,
 1110000033,
 1110000034,
 1110000035,
 1110000036,
 1110000037,
 1110000038,
 1110000039,
 1110000040,
 1110000041,
 1110000042,
 1110000043,
 1110000044,
 1110000045,
 1110000046,
 1110000047,
 1110000048,
 1110000049,
 1110000050,
 1110000051,
 1110000052,
 1110000053,
 1110000054,
 1110000055,
 1110000056,
 1110000057,
 1110000058,
 1110000059,
 1110000060,
 1110000061,
 1110000062,
 1110000063,
 1110000064,
 1110000065,
 1110000066,
 1110000067,
 1110000068,
 1110000069,
 1110000070,
 1110000071,
 1110000072,
 1110000073,
 1110000074,
 1110000075,
 1110000076,

In [33]:
#print(f"sec elapsed: {time.time() - start_time}")

In [41]:
%%time
OD_matrix = gn.calculate_OD(G2, originNodes_list, destinationNodes, fail_value=-1, weight='length')

Wall time: 33min 26s


In [42]:
OD_matrix

array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]])

## calculate accessibility
### For each row, the closest facility is the smallest value in the row

In [43]:
closest_facility_per_origin = OD_matrix.min(axis=1)

In [45]:
results = pd.DataFrame([originNodes_list, closest_facility_per_origin]).transpose()

In [49]:
colName = "travel_time_to_closest_facility"
results.columns = ['osmid', colName]

In [50]:
results[:5]

,osmid,travel_time_to_closest_facility
0,1.110000e+09,-1.0
1,1.110000e+09,-1.0
2,1.110000e+09,-1.0
3,1.110000e+09,-1.0
4,1.110000e+09,-1.0


In [51]:
output2 = pd.merge(pois_meter2, results, on="osmid")

In [52]:
output2

,osmid,VALUE,geometry,travel_time_to_closest_facility
0,1110000000,11.0,POINT (35.60569 23.13432),-1.0
1,1110000001,11.0,POINT (35.61628 23.13432),-1.0
2,1110000002,11.0,POINT (35.59510 23.12619),-1.0
3,1110000003,11.0,POINT (35.60569 23.12619),-1.0
4,1110000004,11.0,POINT (35.61628 23.12619),-1.0
...,...,...,...,...
4783432,1114783432,11.0,POINT (42.68058 -0.19391),-1.0
4783433,1114783433,11.0,POINT (42.69117 -0.19391),-1.0
4783434,1114783434,11.0,POINT (42.66999 -0.20204),-1.0
4783435,1114783435,11.0,POINT (42.68058 -0.20204),-1.0


In [54]:
output2[output2['travel_time_to_closest_facility']==-1]

,osmid,VALUE,geometry,travel_time_to_closest_facility
0,1110000000,11.0,POINT (35.60569 23.13432),-1.0
1,1110000001,11.0,POINT (35.61628 23.13432),-1.0
2,1110000002,11.0,POINT (35.59510 23.12619),-1.0
3,1110000003,11.0,POINT (35.60569 23.12619),-1.0
4,1110000004,11.0,POINT (35.61628 23.12619),-1.0
...,...,...,...,...
4783432,1114783432,11.0,POINT (42.68058 -0.19391),-1.0
4783433,1114783433,11.0,POINT (42.69117 -0.19391),-1.0
4783434,1114783434,11.0,POINT (42.66999 -0.20204),-1.0
4783435,1114783435,11.0,POINT (42.68058 -0.20204),-1.0


In [56]:
output2[output2['travel_time_to_closest_facility']>0]

,osmid,VALUE,geometry,travel_time_to_closest_facility


In [ ]:
#snapped_origins

In [ ]:
#output2 = pd.merge(snapped_origins, results, on="NN")

In [ ]:
#output2[:5]

In [ ]:
#output2['travel_time_to_closest_facility'] = output2['travel_time_to_closest_facility'].astype('int64')

In [ ]:
#output2.dtypes

## save a shapefile..
### Then in QGIS it can be opened and symbolized based on choosing a colored ramp based on the travel_time_to_closest_facility attribute

In [53]:
destinations_gpd = gpd.GeoDataFrame(output2, crs = "epsg:4326", geometry = 'geometry')
destinations_gpd.to_file('horn_of_africa_rural_accessibility6.shp')

<ipython-input-53-b1d14ffa249e>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  destinations_gpd.to_file('horn_of_africa_rural_accessibility6.shp')
